In [2]:
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import requests
import bs4

pd.options.display.float_format = '{:,.3f}'.format

<br> 단순히 스코어가 높다고 무조건 매수했다가 큰 낙폭으로 손해를 볼 수도 있기 때문에 기본적인 필터링이 필요합니다. 

In [215]:
X_outcome = pd.read_pickle('X_outcome.pkl')
X_outcome.head()

,name,return,price_z,volume_z,num_high/close,num_win_market,pct_win_market,return over sector,yhat,max_close,min_close,target
code,,,,,,,,,,,,
238490,힘스,0.997,-1.290,-0.519,0.000,6.000,1.000,1.002,0.222,1.000,0.974,0
037440,희림,0.981,0.144,-0.839,0.000,12.000,1.009,0.970,0.298,1.015,0.924,0
189980,흥국에프엔비,1.004,0.304,-0.555,0.000,14.000,1.002,1.000,0.185,1.019,0.972,0
010240,흥국,0.996,0.962,0.693,0.000,8.000,1.000,1.001,0.229,1.018,0.952,0
024060,흥구석유,1.011,-0.838,-0.590,0.000,15.000,1.006,1.012,0.266,0.989,0.955,0


<br> 각 스코어 구간별 종가 수익율에 따라 'target' (5% 로 익절) 변수가 어떻게 변하는 지 보겠습니다. 높은 스코어 영역에서 종가 수익율이 낮은 구간에서 더 좋은 미래 수익율을 기대할 수 있는 것으로 분석되었습니다.


In [217]:
X_outcome['yhat_rank'] = pd.qcut(X_outcome['yhat'], q=10)
X_outcome['return_rank']  = pd.qcut(X_outcome['return'], q=5)
X_outcome.groupby(['yhat_rank','return_rank'])['target'].mean().unstack()

return_rank,"(0.832, 0.981]","(0.981, 0.992]","(0.992, 1.0]","(1.0, 1.014]","(1.014, 1.3]"
yhat_rank,,,,,
"(0.13, 0.178]",0.250,0.231,0.096,0.242,0.294
"(0.178, 0.191]",0.091,0.147,0.111,0.103,0.143
"(0.191, 0.202]",0.125,0.135,0.056,0.062,0.357
"(0.202, 0.212]",0.241,0.040,0.067,0.188,0.400
"(0.212, 0.223]",0.211,0.098,0.087,0.160,0.296
"(0.223, 0.237]",0.226,0.045,0.207,0.200,0.214
"(0.237, 0.254]",0.167,0.207,0.130,0.043,0.200
"(0.254, 0.28]",0.200,0.233,0.050,0.208,0.194
"(0.28, 0.328]",0.180,0.316,0.167,0.476,0.273


<br> 참고로 groupby 로 데이터를 요약하는 방법은 직관적이나, 각 행과 열의 총계는 보여주지 않는다는 단점이 있습니다. 총계가 보고 싶을 때는 pivot_table 에서 'margins=True' 를 인수로 넣어주면 총계를 볼 수 있습니다.

In [218]:
pd.pivot_table(data = X_outcome, index = 'yhat_rank', columns = 'return_rank', values = 'target', aggfunc='mean', margins=True)

return_rank,"(0.832, 0.981]","(0.981, 0.992]","(0.992, 1.0]","(1.0, 1.014]","(1.014, 1.3]",All
yhat_rank,,,,,,
"(0.13, 0.178]",0.250,0.231,0.096,0.242,0.294,0.191
"(0.178, 0.191]",0.091,0.147,0.111,0.103,0.143,0.119
"(0.191, 0.202]",0.125,0.135,0.056,0.062,0.357,0.119
"(0.202, 0.212]",0.241,0.040,0.067,0.188,0.400,0.176
"(0.212, 0.223]",0.211,0.098,0.087,0.160,0.296,0.163
"(0.223, 0.237]",0.226,0.045,0.207,0.200,0.214,0.185
"(0.237, 0.254]",0.167,0.207,0.130,0.043,0.200,0.154
"(0.254, 0.28]",0.200,0.233,0.050,0.208,0.194,0.185
"(0.28, 0.328]",0.180,0.316,0.167,0.476,0.273,0.267


<br> 스코어가 높고, 종가 수익율이 낮은 종목들의 리스크를 함 보겠습니다. 저가 매수를 해서 그런 것인지 리스크가 적은 것으로 나타났습니다. 따라서 우리는 스코어가 높고, 당일 수익률이 낮은 종목을 매수하겠습니다.

In [195]:
pd.pivot_table(data = X_outcome, index = 'yhat_rank', columns = 'return_rank', values = 'min_close', aggfunc='mean', margins=True)

return_rank,"(0.832, 0.981]","(0.981, 0.992]","(0.992, 1.0]","(1.0, 1.014]","(1.014, 1.3]",All
yhat_rank,,,,,,
"(0.13, 0.178]",0.986,0.983,0.985,0.983,1.000,0.986
"(0.178, 0.191]",0.974,0.969,0.975,0.976,0.976,0.974
"(0.191, 0.202]",0.971,0.977,0.986,0.976,0.995,0.980
"(0.202, 0.212]",0.970,0.974,0.971,0.983,0.978,0.975
"(0.212, 0.223]",0.980,0.966,0.978,0.985,0.984,0.977
"(0.223, 0.237]",0.974,0.976,0.979,0.990,0.981,0.980
"(0.237, 0.254]",0.970,0.980,0.978,0.983,0.983,0.978
"(0.254, 0.28]",0.970,0.978,0.977,0.982,0.965,0.974
"(0.28, 0.328]",0.953,0.960,0.980,0.989,0.966,0.965


<br> 아래와 같이 종목 선택 기준을 설정합니다. 총 13 개의 종목이 선정되었습니다.

In [222]:
c1 = X_outcome['yhat'] >= 0.35      # 모델 점수 0.35 이상
c2 = X_outcome['return'] <= 0.98    # 오늘 주가 수익율이 2% 이하
c3 = X_outcome['price_z'] > 0       # 최근 20일 종가의 평균보다 오늘 종가가 더 높음
selection = X_outcome[c1 & c2 & c3].sort_values(by='yhat', ascending=False)

print(len(selection))

selection

13


,name,return,price_z,volume_z,num_high/close,num_win_market,pct_win_market,return over sector,yhat,max_close,min_close,target,yhat_rank,return_rank
code,,,,,,,,,,,,,,
218150,미래생명자원,0.963,0.324,-0.406,0.000,13.000,1.022,1.010,0.507,1.003,0.940,0,"(0.328, 0.805]","(0.832, 0.981]"
332290,누보,0.966,0.887,1.598,0.000,14.000,1.012,0.974,0.390,1.233,0.975,1,"(0.328, 0.805]","(0.832, 0.981]"
115440,우리넷,0.920,1.778,0.482,0.000,9.000,1.009,0.922,0.386,1.017,0.917,0,"(0.328, 0.805]","(0.832, 0.981]"
024940,PN풍년,0.915,0.602,0.189,0.000,8.000,1.007,0.922,0.381,1.092,0.969,1,"(0.328, 0.805]","(0.832, 0.981]"
214370,케어젠,0.975,1.767,0.424,0.000,6.000,1.010,0.976,0.381,1.154,0.981,1,"(0.328, 0.805]","(0.832, 0.981]"
016600,큐캐피탈,0.894,1.292,1.175,0.000,11.000,1.006,0.893,0.376,1.291,0.994,1,"(0.328, 0.805]","(0.832, 0.981]"
290720,푸드나무,0.977,1.902,3.829,1.000,9.000,1.000,0.920,0.373,1.085,1.000,1,"(0.328, 0.805]","(0.832, 0.981]"
214310,에스엘바이오닉스,0.951,2.202,2.069,0.000,7.000,0.996,0.956,0.372,1.109,0.981,1,"(0.328, 0.805]","(0.832, 0.981]"
289010,아이스크림에듀,0.965,1.620,1.816,1.000,5.000,1.003,0.966,0.368,0.962,0.845,0,"(0.328, 0.805]","(0.832, 0.981]"
